In [1]:
#!/usr/bin/env sage

from sage.all import *
import itertools
import subprocess
import tempfile
import os
import json
import time
import signal
from datetime import datetime
import numpy as np
import pickle
import shutil

# Field definitions
SMALL_PRIME = 65537
STANDARD_PRIME1 = 536870923  # ≈2^29, Supports msolve
STANDARD_PRIME2 = 2147483489  # ≈2^31, Supports msolve
LARGE_PRIME = 9223372036854775783  # ≈2^63, no msolve support

def create_extension_field():
    """Create GF(2^128) extension field"""
    F2 = GF(2)
    R = PolynomialRing(F2, 'x')
    x = R.gen()
    irreducible = x**128 + x**7 + x**2 + x + 1
    return GF(2**128, name='t', modulus=irreducible)

def random_polynomial_system(n, k, field, m, r):
    """Generate random polynomial system with LEX ordering"""
    R = PolynomialRing(field, n, 'x', order='lex')
    x = R.gens()
    ps = []
    
    all_exponents = list(itertools.product(range(k + 1), repeat=n))
    valid_exponents = [exp for exp in all_exponents if sum(exp) <= k]
    
    r0 = min(r, binomial(n+k, k)) if r > 0 else binomial(n+k, k)
    
    for i in range(m):
        poly = 0
        used_indices = set()
        while len(used_indices) < min(r0, len(valid_exponents)):
            idx = randint(0, len(valid_exponents) - 1)
            if idx not in used_indices:
                exponents = valid_exponents[idx]
                coefficient = field.random_element()
                if coefficient != 0:
                    term = coefficient * prod([x[j] ** exponents[j] for j in range(n)])
                    poly += term
                    used_indices.add(idx)
        ps.append(poly)
    
    return ps, R

def cleanup_all_processes():
    """Cleanup to avoid process conflicts"""
    try:
        os.system("pkill -9 -f magma 2>/dev/null")
        os.system("pkill -9 -f Magma 2>/dev/null")
        os.system("rm -f /tmp/magma_* 2>/dev/null")
        os.system("rm -f /tmp/sage_magma_* 2>/dev/null")
        os.system("rm -f /tmp/interface* 2>/dev/null")
        
        import gc
        gc.collect()
        time.sleep(0.5)
    except:
        pass

def benchmark_magma_subprocess(polys, ring, timeout=120):
    """Benchmark Magma using subprocess (F4+FGLM)"""
    try:
        field = ring.base_ring()
        nvars = ring.ngens()
        var_names = ','.join(ring.variable_names())
        
        script_lines = []
        
        if hasattr(field, 'characteristic') and hasattr(field, 'degree'):
            char = field.characteristic()
            deg = field.degree()
            if deg == 1:
                script_lines.append(f"F := GF({char});")
            else:
                script_lines.append(f"F<t> := GF({char}, {deg});")
        else:
            script_lines.append(f"F<t> := GF({field});")
        
        script_lines.extend([
            f"R<{var_names}> := PolynomialRing(F, {nvars}, \"lex\");",
            "polys := [",
        ])
        
        for i, poly in enumerate(polys):
            poly_str = str(poly)
            if i < len(polys) - 1:
                script_lines.append(f"    {poly_str},")
            else:
                script_lines.append(f"    {poly_str}")
        
        script_lines.extend([
            "];",
            "I := ideal<R | polys>;",
            "gb := GroebnerBasis(I);",
            "printf \"%o\\n\", #gb;",
            "quit;"
        ])
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.m', delete=False) as f:
            f.write('\n'.join(script_lines))
            script_file = f.name
        
        start_time = time.time()
        result = subprocess.run(
            ["magma", script_file],
            capture_output=True,
            text=True,
            timeout=timeout
        )
        elapsed = time.time() - start_time
        
        os.remove(script_file)
        
        if result.returncode == 0:
            return elapsed
        else:
            return None
            
    except subprocess.TimeoutExpired:
        try:
            os.remove(script_file)
        except:
            pass
        return f">{timeout}"
    except Exception:
        return None

def benchmark_magma_resultant(polys, ring, timeout=120):
    """Benchmark Magma using Resultant method (only for 2 equations)"""
    if len(polys) != 2:
        return None
        
    try:
        field = ring.base_ring()
        nvars = ring.ngens()
        var_names = ring.variable_names()
        vars_str = ','.join(var_names)
        
        script_lines = []
        
        if hasattr(field, 'characteristic') and hasattr(field, 'degree'):
            char = field.characteristic()
            deg = field.degree()
            if deg == 1:
                script_lines.append(f"F := GF({char});")
            else:
                script_lines.append(f"F<t> := GF({char}, {deg});")
        else:
            script_lines.append(f"F<t> := GF({field});")
        
        script_lines.extend([
            f"R<{vars_str}> := PolynomialRing(F, {nvars}, \"lex\");",
            f"f := {str(polys[0])};",
            f"g := {str(polys[1])};",
            "",
            "// Compute resultant with respect to last variable",
        ])
        
        if nvars == 2:
            script_lines.extend([
                f"res := Resultant(f, g, {var_names[-1]});",
                "// Solve univariate polynomial",
                "if res ne 0 then",
                "    UP := PolynomialRing(F);",
                "    res_uni := UP!res;",
                "    roots := Roots(res_uni);",
                "    printf \"Found %o roots\\n\", #roots;",
                "else",
                "    printf \"Resultant is zero\\n\";",
                "end if;",
            ])
        else:
            script_lines.extend([
                f"// Eliminate {var_names[-1]} first",
                f"res1 := Resultant(f, g, {var_names[-1]});",
                "if res1 ne 0 then",
                "    printf \"Resultant computed\\n\";",
                "else",
                "    printf \"Resultant is zero\\n\";",
                "end if;",
            ])
        
        script_lines.append("quit;")
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.m', delete=False) as f:
            f.write('\n'.join(script_lines))
            script_file = f.name
        
        start_time = time.time()
        result = subprocess.run(
            ["magma", script_file],
            capture_output=True,
            text=True,
            timeout=timeout
        )
        elapsed = time.time() - start_time
        
        os.remove(script_file)
        
        if result.returncode == 0:
            return elapsed
        else:
            return None
            
    except subprocess.TimeoutExpired:
        try:
            os.remove(script_file)
        except:
            pass
        return f">{timeout}"
    except Exception:
        return None

def benchmark_dixon(polys, ring, field_char, m, timeout=120):
    """Benchmark Dixon method"""
    if m <= 1:
        return None
    
    if not shutil.which("dixon"):
        return None
    
    try:
        var_names = [str(v) for v in ring.gens()]
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.dat', delete=False) as f:
            f.write(f"{field_char}\n")
            poly_strs = [str(poly) for poly in polys]
            f.write(", ".join(poly_strs) + "\n")
            eliminate_vars = var_names[:m-1]
            f.write(", ".join(eliminate_vars) + "\n")
            input_file = f.name
        
        start = time.time()
        result = subprocess.run(
            ["dixon", "--silent", input_file],
            capture_output=True,
            text=True,
            timeout=timeout
        )
        elapsed = time.time() - start
        
        try:
            os.remove(input_file)
        except:
            pass
        
        if result.returncode == 0:
            return elapsed
        else:
            if result.stderr:
                print(f"Dixon error: {result.stderr[:200]}")
            return None
            
    except subprocess.TimeoutExpired:
        try:
            os.remove(input_file)
        except:
            pass
        return f">{timeout}"
    except Exception as e:
        print(f"Dixon exception: {str(e)[:200]}")
        return None

def benchmark_msolve(polys, ring, p, n, m, timeout=120):
    """Benchmark msolve elimination (only for supported fields)"""
    if n <= 1 or p > (2**31 - 1):
        return None
        
    try:
        with tempfile.NamedTemporaryFile(mode='w', suffix='.ms', delete=False) as f:
            var_names = [f"x{i}" for i in range(n)]
            f.write(", ".join(var_names) + "\n")
            f.write(f"{p}\n")
            
            for i, poly in enumerate(polys):
                poly_str = str(poly)
                if i < len(polys) - 1:
                    f.write(poly_str + ",\n")
                else:
                    f.write(poly_str + "\n")
            input_file = f.name
        
        eliminate_count = min(n-1, m-1) if m > 1 else n-1
        
        start = time.time()
        result = subprocess.run(
            ["msolve", "-e", str(eliminate_count), "-g", "2", "-f", input_file],
            capture_output=True,
            text=True,
            timeout=timeout
        )
        elapsed = time.time() - start
        
        os.remove(input_file)
        
        if result.returncode == 0:
            return elapsed
        else:
            return None
            
    except subprocess.TimeoutExpired:
        try:
            os.remove(input_file)
        except:
            pass
        return f">{timeout}"
    except Exception:
        return None

def run_single_benchmark(n, d, field, m, r, timeout=120, 
                        include_resultant=False, 
                        include_msolve=False):
    """Run a single benchmark test"""
    set_random_seed(42 + n + d + m)
    polys, ring = random_polynomial_system(n, d, field, m, r)
    
    results = {}
    
    # Get field characteristic
    if hasattr(field, 'characteristic'):
        field_char = field.characteristic()
    else:
        field_char = field
    
    # Benchmark each method
    results['magma'] = benchmark_magma_subprocess(polys, ring, timeout)
    results['dixon'] = benchmark_dixon(polys, ring, field_char, m, timeout)
    
    if include_msolve:
        results['msolve'] = benchmark_msolve(polys, ring, field_char, n, m, timeout)
    else:
        results['msolve'] = None
    
    if include_resultant and m == 2:
        results['magma_resultant'] = benchmark_magma_resultant(polys, ring, timeout)
    else:
        results['magma_resultant'] = None
    
    # Calculate empirical time ratios
    if results['magma'] and results['dixon'] and not isinstance(results['magma'], str) and not isinstance(results['dixon'], str):
        results['dixon_magma_time_ratio'] = results['magma'] / results['dixon']
    else:
        results['dixon_magma_time_ratio'] = None
    
    return results

def format_time_result(result):
    """Format timing result for display"""
    if result is None:
        return "FAIL"
    elif isinstance(result, str) and result.startswith(">"):
        return result + "s"
    else:
        return f"{result:.4f}s"

def run_all_benchmarks():
    """Run complete benchmark suite and save results"""
    import time
    start_time = time.time()
    
    print("="*100)
    print("POLYNOMIAL SOLVER BENCHMARK - DATA COLLECTION")
    print("="*100)
    print(f"Start Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*100)
    
    # Define fields
    small_field = GF(SMALL_PRIME)
    standard_field1 = GF(STANDARD_PRIME1)
    standard_field2 = GF(STANDARD_PRIME2)
    large_prime_field = GF(LARGE_PRIME)
    extension_field = create_extension_field()
    
    fields = {
        'small': small_field,
        'standard1': standard_field1,
        'standard2': standard_field2,
        'large_prime': large_prime_field,
        'extension': extension_field
    }
    
    field_names = {
        'small': f'GF({SMALL_PRIME})',
        'standard1': f'GF({STANDARD_PRIME1})',
        'standard2': f'GF({STANDARD_PRIME2})',
        'large_prime': f'GF({LARGE_PRIME})',
        'extension': 'GF(2^128)'
    }
    
    all_results = {}
    
    for field_key, field in fields.items():
        print(f"\n{'='*80}")
        print(f"TESTING ON {field_names[field_key]}")
        print("="*80)
        
        include_msolve = (field_key == 'small' or 'standard')
        field_results = {}
        

        # TEST: Fixed variables, varying degree
        for n in [2, 3, 4, 5, 6, 7, 8]:
            print(f"\n--- Testing with n={n} variables (square system m=n={n}) ---")
            header = f"{'d':<5} {'Magma':<12} {'Dixon':<12} {'msolve':<12} {'MagmaResultant':<16} {'M/D Ratio':<10}"
            print(header)
            print("-" * len(header))
            
            if n == 2:
                d_range = range(2, 40, 4)
            elif n == 3:
                d_range = range(2, 20, 4)
            elif n == 4:
                d_range = range(2, 10, 2)
            elif n == 5:
                d_range = range(2, 6, 1)
            elif n == 6:
                d_range = range(2, 4, 1)
            else:  # n == 7,8
                d_range = range(2, 3, 1)
            
            test_data = []
            for d in d_range:
                m = n
                r = min(300, binomial(n+d, d))
                timeout = min(3600, 300 * d * n)
                
                results = run_single_benchmark(
                    n, d, field, m, r, timeout,
                    include_resultant=(n==2),
                    include_msolve=include_msolve
                )
                
                test_data.append({
                    'n': n, 'd': d, 'm': m,
                    'results': results
                })
                
                magma_str = format_time_result(results.get('magma'))
                dixon_str = format_time_result(results.get('dixon'))
                msolve_str = format_time_result(results.get('msolve')) if include_msolve else "N/A"
                resultant_str = format_time_result(results.get('magma_resultant')) if n == 2 else "N/A"
                
                ratio_str = "N/A"
                if results.get('dixon_magma_time_ratio') is not None:
                    ratio_str = f"{results['dixon_magma_time_ratio']:.4f}"
                
                print(f"{d:<5} {magma_str:<12} {dixon_str:<12} {msolve_str:<12} {resultant_str:<16} {ratio_str:<10}")
            
            field_results[f'fixed_vars_{n}'] = test_data
        
        all_results[field_key] = field_results
    
    # Save results
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Save as pickle for Python analysis
    pickle_file = f'benchmark_results_{timestamp}.pkl'
    with open(pickle_file, 'wb') as f:
        pickle.dump({
            'results': all_results,
            'field_names': field_names,
            'timestamp': timestamp
        }, f)
    print(f"\n✓ Results saved to {pickle_file}")
    
    # Also save as JSON for portability
    json_file = f'benchmark_results_{timestamp}.json'
    
    def convert_for_json(obj):
        """Convert non-JSON-serializable objects"""
        if isinstance(obj, (np.float32, np.float64)):
            return float(obj)
        elif isinstance(obj, (np.int32, np.int64)):
            return int(obj)
        elif hasattr(obj, 'parent'):  # Sage types
            return float(obj)
        return obj
    
    def deep_convert(obj):
        if isinstance(obj, dict):
            return {k: deep_convert(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [deep_convert(v) for v in obj]
        else:
            return convert_for_json(obj)
    
    with open(json_file, 'w') as f:
        json.dump(deep_convert({
            'results': all_results,
            'field_names': field_names,
            'timestamp': timestamp
        }), f, indent=2)
    print(f"✓ Results also saved as JSON to {json_file}")
    
    print("\n" + "="*100)
    print("DATA COLLECTION COMPLETE")
    print("="*100)
    
    # Calculate and display total time
    end_time = time.time()
    total_time = end_time - start_time
    hours = int(total_time // 3600)
    minutes = int((total_time % 3600) // 60)
    seconds = total_time % 60
    
    print(f"End Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Total Time: {hours}h {minutes}m {seconds:.1f}s")
    print("="*100)
    
    return all_results, field_names, timestamp

if __name__ == "__main__":
    print("\n" + "="*100)
    print("Make sure magma, msolve and dixon is installed and in your PATH")
    print("="*100 + "\n")
    
    # Run benchmarks and save results
    results, names, timestamp = run_all_benchmarks() 
    print(f"\nBenchmark completed at {timestamp}")


Make sure magma, msolve and dixon is installed and in your PATH

POLYNOMIAL SOLVER BENCHMARK - DATA COLLECTION
Start Time: 2025-12-15 18:35:37

TESTING ON GF(65537)

--- Testing with n=2 variables (square system m=n=2) ---
d     Magma        Dixon        msolve       MagmaResultant   M/D Ratio 
------------------------------------------------------------------------
2     0.0653s      0.0074s      0.0332s      0.0326s          8.8650    
6     0.0709s      0.0072s      0.0341s      0.0359s          9.8726    
10    0.0628s      0.0109s      0.0551s      0.0333s          5.7431    
14    0.0846s      0.0204s      0.1690s      0.0299s          4.1415    
18    0.1269s      0.0313s      0.6787s      0.0389s          4.0516    
22    0.2345s      0.0520s      2.1894s      0.0460s          4.5080    
26    0.3849s      0.0631s      5.9865s      0.0502s          6.0987    
30    0.7069s      0.0889s      14.5381s     0.0605s          7.9529    
34    1.6333s      0.1260s      20.0767s     0